# Work Notebook

In [58]:
import pandas as pd

import string

from nltk.tokenize import RegexpTokenizer
import regex as re
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from nltk.corpus import wordnet

In [32]:
pd.set_option('display.max_colwidth', 100)

In [2]:
!ls data

xfiles.csv


In [3]:
df = pd.read_csv('data/xfiles.csv',encoding='unicode_escape' )

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9093 entries, 0 to 9092
Data columns (total 3 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   tweet_text                                          9092 non-null   object
 1   emotion_in_tweet_is_directed_at                     3291 non-null   object
 2   is_there_an_emotion_directed_at_a_brand_or_product  9093 non-null   object
dtypes: object(3)
memory usage: 213.2+ KB


In [5]:
df.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


In [34]:
df['is_there_an_emotion_directed_at_a_brand_or_product'].value_counts(normalize = True)

No emotion toward brand or product    0.592609
Positive emotion                      0.327541
Negative emotion                      0.062692
I can't tell                          0.017158
Name: is_there_an_emotion_directed_at_a_brand_or_product, dtype: float64

In [7]:
df['emotion_in_tweet_is_directed_at'].value_counts().keys()

Index(['iPad', 'Apple', 'iPad or iPhone App', 'Google', 'iPhone',
       'Other Google product or service', 'Android App', 'Android',
       'Other Apple product or service'],
      dtype='object')

In [39]:
df[df['is_there_an_emotion_directed_at_a_brand_or_product'] == "No emotion toward brand or product"].head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product,brand,tweet_clean,target
5,@teachntech00 New iPad Apps For #SpeechTherapy And Communication Are Showcased At The #SXSW Conf...,NaN,No emotion toward brand or product,NaN,New iPad Apps For SpeechTherapy And Communication Are Showcased At The SXSW Conference iear edch...,NaN
16,Holler Gram for iPad on the iTunes App Store - http://t.co/kfN3f5Q (via @marc_is_ken) #sxsw,NaN,No emotion toward brand or product,NaN,Holler Gram for iPad on the iTunes App Store via is ken sxsw,NaN
32,"Attn: All #SXSW frineds, @mention Register for #GDGTLive and see Cobra iRadar for Android. {link}",NaN,No emotion toward brand or product,NaN,Attn All SXSW frineds Register for GDGTLive and see Cobra iRadar for Android link,NaN
33,Anyone at #sxsw want to sell their old iPad?,NaN,No emotion toward brand or product,NaN,Anyone at sxsw want to sell their old iPad,NaN
34,Anyone at #SXSW who bought the new iPad want to sell their older iPad to me?,NaN,No emotion toward brand or product,NaN,Anyone at SXSW who bought the new iPad want to sell their older iPad to me,NaN


In [8]:
products = {'Apple':'Apple','iPad':'Apple', 'iPad or iPhone App':'Apple','iPhone':'Apple','Other Apple product or service':'Apple',
            'Other Google product or service':'Google','Android App':'Google', 'Android':'Google', 'Google':'Google' }

In [9]:
df['brand'] = df['emotion_in_tweet_is_directed_at'].map(products)

In [10]:
df.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product,brand
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion,Apple
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion,Apple
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion,Apple
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion,Apple
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion,Google


In [11]:
df.brand.value_counts()

Apple     2409
Google     882
Name: brand, dtype: int64

In [12]:
(df.brand.isna()).sum()

5802

In [28]:
df[df['emotion_in_tweet_is_directed_at'].isna()]

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product,brand,tweet_clean
5,@teachntech00 New iPad Apps For #SpeechTherapy...,NaN,No emotion toward brand or product,NaN,New iPad Apps For SpeechTherapy And Communicat...
16,Holler Gram for iPad on the iTunes App Store -...,NaN,No emotion toward brand or product,NaN,Holler Gram for iPad on the iTunes App Store v...
32,"Attn: All #SXSW frineds, @mention Register fo...",NaN,No emotion toward brand or product,NaN,Attn All SXSW frineds Register for GDGTLive an...
33,Anyone at #sxsw want to sell their old iPad?,NaN,No emotion toward brand or product,NaN,Anyone at sxsw want to sell their old iPad
34,Anyone at #SXSW who bought the new iPad want ...,NaN,No emotion toward brand or product,NaN,Anyone at SXSW who bought the new iPad want to...
...,...,...,...,...,...
9087,"@mention Yup, but I don't have a third app yet...",NaN,No emotion toward brand or product,NaN,Yup but I don t have a third app yet I m on An...
9089,"Wave, buzz... RT @mention We interrupt your re...",NaN,No emotion toward brand or product,NaN,Wave buzz RT We interrupt your regularly sched...
9090,"Google's Zeiger, a physician never reported po...",NaN,No emotion toward brand or product,NaN,Google s Zeiger a physician never reported pot...
9091,Some Verizon iPhone customers complained their...,NaN,No emotion toward brand or product,NaN,Some Verizon iPhone customers complained their...


In [14]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [15]:
tokenizer = RegexpTokenizer("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)")

In [16]:
def athash_remover(x):
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x).split())

https://stackoverflow.com/questions/8376691/how-to-remove-hashtag-user-link-of-a-tweet-using-regular-expression

In [17]:
df.drop(6, inplace= True)

In [18]:
df['tweet_text'].isna().sum()

0

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9092 entries, 0 to 9092
Data columns (total 4 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   tweet_text                                          9092 non-null   object
 1   emotion_in_tweet_is_directed_at                     3291 non-null   object
 2   is_there_an_emotion_directed_at_a_brand_or_product  9092 non-null   object
 3   brand                                               3291 non-null   object
dtypes: object(4)
memory usage: 355.2+ KB


In [20]:
df['tweet_clean'] = df['tweet_text'].apply(athash_remover, )

In [21]:
df.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product,brand,tweet_clean
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion,Apple,I have a 3G iPhone After 3 hrs tweeting at RIS...
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion,Apple,Know about Awesome iPad iPhone app that you ll...
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion,Apple,Can not wait for iPad 2 also They should sale ...
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion,Apple,I hope this year s festival isn t as crashy as...
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion,Google,great stuff on Fri SXSW Marissa Mayer Google T...


In [49]:
df['target'] = df['is_there_an_emotion_directed_at_a_brand_or_product'].map({'Negative emotion':1, 'Positive emotion':2, 
                                                          'No emotion toward brand or product': 0})

In [56]:
df.dropna(axis = 0, subset = ['target'], inplace = True)

In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8936 entries, 0 to 9092
Data columns (total 6 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   tweet_text                                          8936 non-null   object 
 1   emotion_in_tweet_is_directed_at                     3282 non-null   object 
 2   is_there_an_emotion_directed_at_a_brand_or_product  8936 non-null   object 
 3   brand                                               3282 non-null   object 
 4   tweet_clean                                         8936 non-null   object 
 5   target                                              8936 non-null   float64
dtypes: float64(1), object(5)
memory usage: 488.7+ KB


In [61]:
df.to_csv('data/clean_tweets.csv')

In [ ]:
X = df.tweet_text
y = df.target

X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 10)